In [6]:
import numpy as np
import pandas as pd

In [17]:
df_off = pd.read_csv('labeled_data.csv')
df_tweet = pd.read_csv('train.csv')

df_tweet.drop('id', axis=1, inplace=True)
df_off.drop(['Unnamed: 0', 'count', 'hate_speech',
					'offensive_language', 'neither'], axis=1, inplace=True)

df_off[df_off['class']==0]['class']=1
df_off[df_off['class']==0]

df_off["class"].replace({0: 1}, inplace=True)

df_off[df_off['class']==0]
df_off["class"].replace({2: 0}, inplace=True)
df_off.rename(columns={'class': 'label'}, inplace=True)




C:\Users\DELLG3~1\AppData\Local\Temp/ipykernel_13664/1235827781.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_off[df_off['class']==0]['class']=1


In [18]:
frame=[df_tweet,df_off]
df2 = pd.concat(frame)

In [19]:
df2.head()

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [27]:
import nltk
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')

import re
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to C:\Users\DELL
[nltk_data]     G3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):

    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)  
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    return text
df2['tweet'] = df2['tweet'].apply(clean_text)

In [30]:
df2.head(5)

,label,tweet
0,0,user father dysfunctional selfish drags kids d...
1,0,user user thanks #lyft credit cant use cause d...
2,0,bihday majesty
3,0,#model love u take u time ur
4,0,factsguide society #motivation


In [32]:
import tensorflow
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

In [33]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer2 = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer2.fit_on_texts(df2['tweet'].values)
word_index = tokenizer2.word_index
print('Found %s unique tokens.' % len(word_index))

Found 67728 unique tokens.


In [39]:
load_model = keras.models.load_model("./hate&speech_lstm2.h5")

In [43]:
test = 'i hate my country'
test = [clean_text(test)]
seq = tokenizer2.texts_to_sequences(test)
print('seq: ',seq)
padded = sequence.pad_sequences(seq, maxlen=250)
print('padded: ', padded)
pred = load_model.predict(padded)
print('pred: ', pred)
result = [pred]
if pred < 0.45:
	print('no hate')
	res_text = "no hate"
else:
	print('hate')
	res_text = "hate and abusive"

[[71, 485]]
seq:  [[]]
padded:  [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
pred:  [[0.19769916]]
no hate
